# 텍스트 데이터 토큰화 진행 후 파일 형태로 저장하기
- okt
- mecab
- sentencepiece


In [2]:
import os
import pandas as pd
import tensorflow
import numpy as np
import matplotlib
import os
import re
import sentencepiece as spm
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
test_data = pd.read_json("~/data/test.json").T
train_data_path ="~/data/train.csv"
train_data = pd.read_csv(train_data_path, index_col=0)
train_data.head()

,class,conversation
idx,,
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [3]:
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [4]:
df = train_data.copy()

In [5]:
df.shape

(3950, 2)

In [6]:
df.isnull().sum()

class           0
conversation    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3950 entries, 0 to 3949
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class         3950 non-null   object
 1   conversation  3950 non-null   object
dtypes: object(2)
memory usage: 92.6+ KB


In [8]:
len(df['conversation'].values.sum())

894955

In [9]:
import re

def clean_text(text):
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)  # 한글, 영문, 공백 제외한 문자 제거
    text = text.strip()  # 양 끝 공백 제거
    return text


In [10]:
df['conversation'] = df['conversation'].apply(clean_text)
df['conversation'] = df['conversation'].str.replace('\n', ' ')

df.head()

,class,conversation
idx,,
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자...
1,협박 대화,길동경찰서입니다 시 분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만 더 얘기한...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지...
3,갈취 대화,어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 얘 돈 좀 있나...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발...


In [11]:
# 0, 3, 1, 2
df['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화'], dtype=object)

In [12]:
# corpus 생성
corpus_0 = df[df['class']=='협박 대화']['conversation'].tolist()
corpus_1 = df[df['class']=='갈취 대화']['conversation'].tolist()
corpus_2 = df[df['class']=='직장 내 괴롭힘 대화']['conversation'].tolist()
corpus_3 = df[df['class']=='기타 괴롭힘 대화']['conversation'].tolist()

In [11]:
from konlpy.tag import Okt, Mecab
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import operator

# 한국어 형태소 분석기 초기화
okt = Okt()
mecab = Mecab()

def tokenize(text):
    tokens = okt.morphs(text)  # 문장을 형태소 단위로 분리
    return tokens

# 불용어 리스트 정의
stop_words = [
    "은", "는", "이", "가", "을", "를", "으로", "에", "에서",
    "하다", "하는", "된", "나", "고", "와", "과", "너무", "있는", "같은", 
    "하는", "것", "그", "그리고", "하는", "하는", "말", "생각", "다", "그런",
    "저", "제", "네", "들", "이다", "입니다", "한다", "합니다", "돼", "이런",
    "저런", "그런", "할", "수z", "있는", "또는", "아니", "합니다", "있다", "왜",
    "여기", "저기", "그렇게", "그러니까", "뭐", "하나", "때", "수", "등", "아주",
    "너", "내", "야", "안", "니", "도"
    # 추가적인 불용어 단어 추가 가능
]

def return_words(corpus):
    # filter stop words
    vect = CountVectorizer(tokenizer=tokenize, stop_words=stopwords, analyzer='word')
    bow_vect = vect.fit_transform(corpus)
    word_list = vect.get_feature_names()  # 어떤 단어들이 들어가있는지 확인
    count_list = bow_vect.toarray().sum(axis=0)  # 인덱스순으로 몇개의 단어가 있는지
    word_count_dict = dict(zip(word_list, count_list))  # word, count를 key value로 묶는 것

    # 빈도수가 높은 순으로 정렬
    sorted_word_count = sorted(word_count_dict.items(), key=operator.itemgetter(1), reverse=True)
    print(sorted_word_count)


In [14]:
#'협박 대화'
return_words(corpus_0)

[('못', 627), ('아', 473), ('제발', 462), ('해', 442), ('만', 431), ('지금', 399), ('거', 393), ('잘', 377), ('로', 356), ('한', 347), ('진짜', 341), ('우리', 327), ('이야', 308), ('사람', 301), ('그래', 289), ('한테', 289), ('게', 287), ('알', 281), ('무슨', 275), ('더', 259), ('요', 257), ('그냥', 252), ('죽여', 248), ('까지', 243), ('죄송합니다', 241), ('좀', 240), ('돈', 228), ('지', 224), ('그럼', 218), ('하면', 217), ('정말', 202), ('어떻게', 201), ('새끼', 195), ('집', 188), ('당신', 172), ('랑', 172), ('봐', 172), ('가족', 171), ('난', 169), ('칼', 167), ('있어', 166), ('넌', 164), ('신고', 161), ('아니야', 161), ('소리', 159), ('하고', 158), ('당장', 157), ('면', 154), ('시간', 149), ('일', 149), ('이제', 148), ('살려주세요', 143), ('딸', 141), ('애', 138), ('하', 138), ('어디', 133), ('죽', 131), ('서', 130), ('빨리', 129), ('전', 129), ('한번', 129), ('나도', 128), ('이렇게', 126), ('죽어', 123), ('하지', 123), ('어', 121), ('줄', 120), ('없어', 118), ('의', 118), ('경찰', 117), ('날', 117), ('그게', 116), ('뭘', 115), ('오늘', 115), ('버릴거야', 113), ('같이', 102), ('앞', 100), ('정신', 100), ('눈', 98)

In [15]:
# '갈취 대화'
return_words(corpus_1)

[('돈', 1477), ('거', 737), ('아', 733), ('좀', 614), ('만', 532), ('요', 486), ('진짜', 463), ('만원', 404), ('그럼', 387), ('한테', 360), ('지금', 348), ('게', 331), ('그래', 319), ('한', 295), ('봐', 294), ('못', 292), ('없어', 271), ('이야', 270), ('내놔', 269), ('로', 266), ('그냥', 251), ('제발', 237), ('줄', 230), ('해', 230), ('없어요', 229), ('잘', 228), ('무슨', 227), ('알', 213), ('우리', 213), ('오늘', 207), ('이번', 207), ('까지', 206), ('빨리', 205), ('엄마', 198), ('더', 196), ('줘', 191), ('친구', 188), ('나도', 176), ('어', 170), ('정말', 160), ('그건', 157), ('집', 153), ('그거', 152), ('응', 149), ('사람', 148), ('지', 148), ('어떻게', 146), ('있어', 144), ('새끼', 140), ('면', 138), ('님', 128), ('랑', 123), ('한번', 121), ('얼마', 120), ('저번', 117), ('돼요', 116), ('일', 114), ('그게', 113), ('빌려', 113), ('인데', 113), ('하면', 112), ('근데', 111), ('내일', 111), ('아니야', 111), ('전', 111), ('좋은', 111), ('래', 108), ('하고', 108), ('신고', 104), ('알겠어', 103), ('저희', 103), ('거기', 102), ('미안해', 102), ('하', 102), ('서', 101), ('달라', 100), ('당장', 100), ('형', 100), ('이제', 98

In [16]:
#'직장 내 괴롭힘 대화'
return_words(corpus_2)

[('죄송합니다', 877), ('님', 784), ('아', 722), ('일', 638), ('좀', 607), ('못', 540), ('씨', 500), ('거', 488), ('요', 470), ('대리', 458), ('회사', 455), ('해', 420), ('지금', 378), ('잘', 363), ('오늘', 349), ('하고', 348), ('한', 336), ('부장', 322), ('무슨', 288), ('로', 282), ('김', 279), ('그럼', 277), ('만', 274), ('사람', 269), ('까지', 248), ('우리', 240), ('그래', 237), ('과장', 230), ('일이', 228), ('자네', 228), ('한테', 218), ('알', 217), ('아닙니다', 216), ('이번', 215), ('하겠습니다', 213), ('게', 211), ('팀', 211), ('이야', 208), ('봐', 191), ('업무', 189), ('지', 185), ('진짜', 185), ('말씀', 184), ('이렇게', 180), ('시간', 179), ('그냥', 176), ('하면', 176), ('내일', 171), ('휴가', 170), ('더', 166), ('알겠습니다', 159), ('그게', 155), ('인데', 152), ('그건', 149), ('해서', 143), ('그래도', 137), ('서', 136), ('커피', 135), ('다시', 133), ('랑', 132), ('어떻게', 132), ('면', 131), ('빨리', 129), ('정말', 129), ('다른', 128), ('하', 128), ('같이', 122), ('같습니다', 121), ('부터', 121), ('앞', 120), ('요즘', 120), ('장님', 120), ('제대로', 119), ('애', 114), ('집', 114), ('퇴근', 112), ('걸', 105), ('부탁', 10

In [17]:
# '기타 괴롭힘 대화'
return_words(corpus_3)

[('아', 1076), ('좀', 598), ('못', 584), ('거', 520), ('진짜', 477), ('요', 379), ('해', 355), ('한테', 354), ('그래', 333), ('우리', 325), ('잘', 324), ('그냥', 321), ('무슨', 305), ('봐', 294), ('만', 285), ('이야', 273), ('지금', 260), ('한', 257), ('게', 249), ('애', 245), ('아니야', 237), ('사람', 235), ('랑', 231), ('죄송합니다', 229), ('응', 219), ('그럼', 217), ('새끼', 215), ('로', 211), ('하고', 208), ('님', 203), ('어', 191), ('넌', 181), ('고객', 176), ('엄마', 176), ('얘', 170), ('친구', 170), ('제발', 168), ('돈', 165), ('지', 165), ('집', 160), ('뭘', 157), ('어떻게', 155), ('냄새', 152), ('더', 149), ('손님', 148), ('난', 147), ('그만해', 146), ('알', 144), ('면', 143), ('하', 142), ('하지', 142), ('이렇게', 141), ('얼굴', 140), ('하지마', 140), ('인데', 132), ('오늘', 130), ('라고', 127), ('기분', 125), ('환불', 125), ('그만', 124), ('돼지', 124), ('까지', 121), ('서', 120), ('그게', 116), ('근데', 116), ('미안해', 115), ('빨리', 113), ('냐', 112), ('옷', 112), ('나도', 107), ('니까', 107), ('일', 107), ('소리', 106), ('장애인', 106), ('같이', 102), ('걸', 101), ('없어', 100), ('뭔', 98), ('있어', 98

In [18]:
df['okt'] = df['conversation'].apply(tokenize)


In [19]:
def tokenize(text):
    tokens = mecab.morphs(text)  # 문장을 형태소 단위로 분리
    return tokens

df['mecab'] = df['conversation'].apply(tokenize)

In [20]:
df

,class,conversation,okt,mecab
idx,,,,
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자...,"[지금, 너, 스스로, 를, 죽여, 달라, 고, 애원, 하는, 것, 인가, 아닙니다...","[지금, 너, 스스로, 를, 죽여, 달, 라고, 애원, 하, 는, 것, 인가, 아닙..."
1,협박 대화,길동경찰서입니다 시 분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만 더 얘기한...,"[길동, 경찰서, 입니다, 시, 분, 마트, 에, 폭발물, 을, 설치, 할거다, 네...","[길동, 경찰서, 입니다, 시, 분, 마트, 에, 폭발물, 을, 설치, 할, 거, ..."
2,기타 괴롭힘 대화,너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지...,"[너, 되게, 귀여운거, 알, 지, 나, 보다, 작은, 남자, 는, 첨, 봤어, 그...","[너, 되게, 귀여운, 거, 알, 지, 나, 보다, 작, 은, 남자, 는, 첨, 봤..."
3,갈취 대화,어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 얘 돈 좀 있나...,"[어이, 거기, 예, 너, 말, 이야, 너, 이리, 오라, 고, 무슨, 일, 너, ...","[어, 이, 거기, 예, 너, 말, 이, 야, 너, 이리, 오, 라고, 무슨, 일,..."
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발...,"[저기, 요, 혹시, 날, 이, 너무, 뜨겁잖아요, 저희, 회사, 에서, 이, 선크...","[저기, 요, 혹시, 날, 이, 너무, 뜨겁, 잖아요, 저희, 회사, 에서, 이, ..."
...,...,...,...,...
3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐 내 머리가 뭐 밥먹으면 대가리만 크냐 너는 아니 ...,"[준, 하야, 넌, 대가리, 가, 왜, 이렇게, 크냐, 내, 머리, 가, 뭐, 밥,...","[준, 하, 야, 넌, 대가리, 가, 왜, 이렇게, 크, 냐, 내, 머리, 가, 뭐..."
3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 억만 보내 예 선생님 ...,"[내, 가, 지금, 너, 아들, 김길준, 데리, 고, 있어, 살리고, 싶으면, 계좌...","[내, 가, 지금, 너, 아들, 김길준, 데리, 고, 있, 어, 살리, 고, 싶, ..."
3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지 왜 그래 들리겠어 들리라...,"[나, 는, 씨, 같은, 사람, 보면, 참, 신기하더라, 어떻게, 저렇게, 살, 지...","[나, 는, 씨, 같, 은, 사람, 보, 면, 참, 신기, 하, 더라, 어떻게, 저..."


In [7]:
# 불용어 제거
def preprocess_text(text):
    tokens = okt.morphs(text)  # 형태소 분석
    tokens = [token for token in df[''] if token not in stop_words]  # 불용어 제거
    return tokens

In [30]:
df.to_csv('data.csv', index=False)

In [31]:
df['okt'] = [token for token in df['okt'] if token not in stop_words and len(token)>1]


In [32]:
df['mecab'] = [token for token in df['mecab'] if token not in stop_words and len(token)>1]


In [33]:
df.to_csv('data2.csv', index=False)

In [5]:
df = pd.read_json('./data/test.json').T
df

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [12]:
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\n', ' ')
df['okt'] = df['text'].apply(tokenize)
def tokenize(text):
    tokens = mecab.morphs(text)  # 문장을 형태소 단위로 분리
    return tokens

df['mecab'] = df['text'].apply(tokenize)
df['mecab'] = [token for token in df['mecab'] if token not in stop_words and len(token)>1]
df['okt'] = [token for token in df['okt'] if token not in stop_words and len(token)>1]


In [13]:
df

,text,okt,mecab
t_000,아가씨 담배한갑주소 네 원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그...,"[아가씨, 담배, 한, 갑주소, 네, 원, 입니다, 어, 네, 지갑, 어디, 갔지,...","[아가씨, 담배, 한갑, 주소, 네, 원, 입니다, 어, 네, 지갑, 어디, 갔, ..."
t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 ...,"[우리, 팀, 에서, 다른, 팀, 으로, 갈, 사람, 없나, 그럼, 영, 지씨, 가...","[우리, 팀, 에서, 다른, 팀, 으로, 갈, 사람, 없, 나, 그럼, 영지, 씨,..."
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...,"[너, 오늘, 그게, 뭐, 야, 네, 제, 가, 뭘, 잘, 못, 했나요, 제대로, ...","[너, 오늘, 그게, 뭐, 야, 네, 제, 가, 뭘, 잘못, 했, 나요, 제대로, ..."
t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...,"[아무튼, 앞, 으로, 니, 가, 내, 와이파이, 야, 응, 와이파이, 온, 켰어,...","[아무튼, 앞, 으로, 니, 가, 내, 와이파이, 야, 응, 와이파이, 온, 켰, ..."
t_005,그러니까 빨리 말해 선생님 제발 살려주십시오 비밀번호 틀릴 때마다 손톱 하나씩 뺀...,"[그러니까, 빨리, 말, 해, 선생님, 제발, 살려주십시오, 비밀번호, 틀릴, 때,...","[그러니까, 빨리, 말, 해, 선생, 님, 제발, 살려, 주, 십시오, 비밀, 번호..."
...,...,...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...,"[미나, 씨, 휴가, 결제, 올, 리기, 전, 에, 저, 랑, 상의, 하, 라고, ...","[미나, 씨, 휴가, 결제, 올리, 기, 전, 에, 저, 랑, 상의, 하, 라고, ..."
t_496,교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 ...,"[교수, 님, 제, 논문, 에, 제, 이름, 이, 없나요, 아, 무슨, 논문, 말,...","[교수, 님, 제, 논문, 에, 제, 이름, 이, 없, 나요, 아, 무슨, 논문, ..."
t_497,야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니...,"[야, 너, 네, 저, 요, 그래, 너, 왜, 요, 돈좀, 줘, 봐, 돈, 없어요,...","[야, 너, 네, 저, 요, 그래, 너, 왜, 요, 돈, 좀, 줘, 봐, 돈, 없,..."
t_498,야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...,"[야, 너, 빨리, 안, 뛰어와, 너, 이, 환자, 제대로, 봤어, 안, 봤어, 어...","[야, 너, 빨리, 안, 뛰어와, 너, 이, 환자, 제대로, 봤, 어, 안, 봤, ..."


In [14]:
df.to_csv('test.csv', index=False)